In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
URL = 'https://www.va.gov/directory/guide/allstate.asp'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [6]:
URL = 'https://crimegrade.org/safest-places-in-colorado-springs-co-metro/'
page = requests.get(URL)
page
# soup = BeautifulSoup(page.content, "html.parser")
# soup

<Response [403]>

In [110]:
# 1. get all states
def getStates():
    states = []
    URL = 'https://www.va.gov/directory/guide/allstate.asp'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    for t in soup.find_all("a", {"class": "reglink"}):
        states.append(t['href'][-2:])
    return states

In [121]:
# 2. get all address and phone
def getInfo(state):
    # get table from website
    URL = 'https://www.va.gov/directory/guide/fac_list_by_state.cfm?State={}'.format(state)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find('table')
    
    # remove theads
    for div in table.find_all("thead"):
        div.decompose()

    # convert to df
    df = pd.read_html(str(table))[0]
    df.columns = ['Station ID', 'Facility', 'Address', 'Phone', 'empty']
    df.drop('empty', axis=1, inplace=True)
    df.dropna(subset = ["Station ID"], inplace=True)
    df["State"] = state
    return df


In [122]:
states_list = getStates()
df_list = []

for state in states_list:
    df_list.append(getInfo(state))
    
df = pd.concat(df_list)
df.to_csv('VA_facilities.csv')